# Validate the LightPFP models
* Compare the MD trajectories produced by PFP and LightPFP

In [ ]:
light_pfp_dir = "light_pfp_md_small" # For LARGE model, "light_pfp_md_large"
pfp_dir = "pfp_md"
maters = ["anatase_TiO2_Ni_0", "anatase_TiO2_Ni_1", "anatase_TiO2_Ni_2", "anatase_TiO2_Ni_3", "anatase_TiO2_Ni_4", "anatase_TiO2_Ni_5"]
temp = [1200, 1400, 1600]

## Density

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ase import units
from ase.io import Trajectory


def get_density(atoms):
    return atoms.get_masses().sum() / units.kg / atoms.get_volume() * 1e27


for mater in maters:
    pfp_density = [
        [get_density(atoms) for atoms in Trajectory(f"{pfp_dir}/pfp_md_{mater}_{t}.traj")]
        for t in temp
    ]
    lpfp_density = [
        [get_density(atoms) for atoms in Trajectory(f"{light_pfp_dir}/light_pfp_md_{mater}_{t}.traj")]
        for t in temp
    ]
    
    
    time = np.arange(len(pfp_density[0]))/10
    
    for t, pfp, lpfp, c in zip(temp, pfp_density, lpfp_density, ["b", "r", "g", "y", "k", "c"]):
        plt.plot(time, pfp, c=c, label=f"PFP {t}K, {np.mean(pfp[-400:]):.3f}")
        plt.plot(time, lpfp, c=c, ls="--", label=f"LightPFP {t}K, {np.mean(lpfp[-400:]):.3f}")
        
    plt.xlabel("Time (ps)")
    plt.ylabel("Density (g/cm^3)")
    plt.legend()
    plt.title(mater)
    plt.savefig(f"{mater}_density.png")

## 2. Radial distribution function

In [ ]:
from light_pfp_evaluate.md import plot_rdf
from ase.io import Trajectory

for mater in maters:
    plot_rdf(
        temp,
        [
            Trajectory(f"{pfp_dir}/pfp_md_{mater}_{t}.traj")[-100:]
            for t in temp
        ],
        [
            Trajectory(f"{light_pfp_dir}/light_pfp_md_{mater}_{t}.traj")[-100:]
            for t in temp
        ],
        f"{mater}_rdf.png"
    )

## 3. Diffusion behavior¶
* Mean squared displacement (MSD)
  * Note: The sliding movement of two surfaces are observed in the MD trajectories (both PFP and LightPFP). To eliminate the effects on MSE, the displacement of atoms are relative to the center of mass of each slab.
* Diffusion coefficient
  * Diffusion coefficient is calculated from the slope of MSD
* Diffusion activation energy
  * Activation energy is calculation from the Arrinhus plot of diffusion coefficient

In [ ]:
import numpy as np
from ase.io import Trajectory

def get_msd_element(traj, element, ref=None):
    numbers = traj[0].get_atomic_numbers()
    pos = np.array([atoms[numbers==element].get_positions() for atoms in traj])
    if ref is None:
        ref = [element]

    # Get the COM of slab
    ref_ind = np.array([i in ref for i in numbers])
    ref_pos = np.array([atoms[ref_ind].get_positions() for atoms in traj])
    com = np.mean(ref_pos, axis=1, keepdims=True)

    # Relative position against COM 
    pos -= com
    msd = [np.mean(np.sum((pos[i+1:] - pos[:-(i+1)])**2, axis=2)) for i in range(len(pos)-1)]
    return msd


def get_diffusion_coef(msd, time_interval):
    time = np.arange(len(msd)) * time_interval
    D = np.polyfit(time, msd, 1)[0] / 6 *1e-1 # cm^2/s
    return D


def get_activation_energy(temperatures, diffusion_coefficients):
    for d in diffusion_coefficients:
        if d < 0.0:
            return None
    k_B = 8.617333262145e-5
    ln_D = [np.log(d) for d in diffusion_coefficients]
    inv_T = [1 /t for t in temperatures]
    slope, intercept = np.polyfit(inv_T, ln_D, 1)
    E_a = -slope * k_B  # in eV
    return E_a


In [ ]:
import matplotlib.pyplot as plt

temp_inv = [1000/t for t in temp]
for mater in maters:
    elements = ["Ni", "O", "Ti"]
    pfp_msd_dict = {e: [] for e in elements}
    lpfp_msd_dict = {e: [] for e in elements}

    for t in temp:
        pfp_msd_dict["Ni"].append(get_msd_element(Trajectory(f"{pfp_dir}/pfp_md_{mater}_{t}.traj")[150::10], 28))
        lpfp_msd_dict["Ni"].append(get_msd_element(Trajectory(f"{light_pfp_dir}/light_pfp_md_{mater}_{t}.traj")[150::10], 28))
        pfp_msd_dict["O"].append(get_msd_element(Trajectory(f"{pfp_dir}/pfp_md_{mater}_{t}.traj")[150::10], 8, [22, 8]))
        lpfp_msd_dict["O"].append(get_msd_element(Trajectory(f"{light_pfp_dir}/light_pfp_md_{mater}_{t}.traj")[150::10], 8, [22, 8]))
        pfp_msd_dict["Ti"].append(get_msd_element(Trajectory(f"{pfp_dir}/pfp_md_{mater}_{t}.traj")[150::10], 22, [22, 8]))
        lpfp_msd_dict["Ti"].append(get_msd_element(Trajectory(f"{light_pfp_dir}/light_pfp_md_{mater}_{t}.traj")[150::10], 22, [22, 8]))
    
    pfp_d_dict = {e: [] for e in elements}
    lpfp_d_dict = {e: [] for e in elements}
    for e in elements:
        for pfp_msd, lpfp_msd in zip(pfp_msd_dict[e], lpfp_msd_dict[e]):
            pfp_d_dict[e].append(get_diffusion_coef(pfp_msd, 1000))
            lpfp_d_dict[e].append(get_diffusion_coef(lpfp_msd, 1000))
    
    fig, ax = plt.subplots(2, 3, figsize=[15, 12])
    for n, e in enumerate(elements):
        pfp_msd = pfp_msd_dict[e]
        lpfp_msd = lpfp_msd_dict[e]
        pfp_d = pfp_d_dict[e]
        lpfp_d = lpfp_d_dict[e]
        for i, (t, c) in enumerate(zip(temp, ["b", "r", "g", "y", "k", "c"])):
            ax[0][n].plot(np.arange(len(pfp_msd[i])), pfp_msd[i], label=f"PFP {t}K {pfp_d[i]*10**6:.3f}", c=c)
            ax[0][n].plot(np.arange(len(lpfp_msd[i])), lpfp_msd[i], label=f"LightPFP {t}K {lpfp_d[i]*10**6:.3f}", c=c, linestyle="--")
        ax[0][n].set_xlabel("time (ps)")
        ax[0][n].set_ylabel("msd")
        ax[0][n].set_title(e)
        ax[0][n].legend()
    
    for n, e in enumerate(elements):
        pfp_d = pfp_d_dict[e]
        lpfp_d = lpfp_d_dict[e]
        pfp_log_d = [np.log10(d) for d in pfp_d]
        lpfp_log_d = [np.log10(d) for d in lpfp_d]
        Ea_pfp = get_activation_energy(temp, pfp_d)
        Ea_lpfp = get_activation_energy(temp, lpfp_d)
        if Ea_pfp is None:
            label = "PFP"
        else:
            label = f"PFP {Ea_pfp:.3f}eV"
        ax[1][n].scatter(temp_inv, pfp_log_d, label=label)
        if Ea_lpfp is None:
            label = "LightPFP"
        else:
            label = f"LightPFP {Ea_lpfp:.3f}eV"
        ax[1][n].scatter(temp_inv, lpfp_log_d, label=label)
        ax[1][n].set_xlabel("1/1000 K")
        ax[1][n].set_ylabel("log(D) (cm^2/s)")
        ax[1][n].set_title(e)
        ax[1][n].legend()
    plt.savefig(f"{mater}_diffusion.png")